Install needed libraries

In [1]:
#1
%pip install requests
%pip install PyGithub
%pip install openai
%pip install --upgrade openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Imports

In [2]:
#2
from openai import AzureOpenAI
import json
import requests
import re
from dotenv import load_dotenv
import os

Function to check if input is already a GitHub repo

In [3]:
#3
def is_github_repo_link(env):
    return env.startswith("https://github.com/") or env.startswith("http://github.com/")

Prompt for user

In [4]:
#4
def input_function():
    prompt = input("Enter a name of the environment or link to GitHub repo: ")
    flag = True
    if is_github_repo_link(prompt) == False:
        flag = False
    return prompt, flag

OpenAI API for getting repositories

In [41]:
#5
def generate_github_repos(prompt, model="gpt-4o"):
    load_dotenv()
    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION")
    )
    system_prompt = "You are an AI assistant that suggests relevant GitHub repositories based on a project description."
    user_prompt = f"Suggest GitHub repositories for a project that involves the following technologies: {prompt}. Provide only the URLs of the repositories."

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt} 
        ]
    )

    suggested_repos = response.choices[0].message.content.strip()
    repo_urls = re.findall(r'(https?://github\.com/[^\s]+)', suggested_repos)
    return repo_urls

Function for filtering repositories

In [35]:
#6
def fetch_repo_details(repo_url):
    github_api_key = os.getenv("GITHUB_API_KEY")  

    repo_api_url = repo_url.replace("https://github.com", "https://api.github.com/repos")

    try:
        headers = {
            'Authorization': f'Bearer {github_api_key}',
            'Accept': 'application/vnd.github.v3+json'  
        }
        response = requests.get(repo_api_url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch repository details from {repo_api_url}. Status code: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error while trying to fetch repository details from {repo_api_url}: {e}")
        return None

def get_top_repos(repo_urls, top_n=3):
    repo_details = []
    for repo_url in repo_urls:
        details = fetch_repo_details(repo_url)
        if details:
            repo_details.append(details)
    sorted_repos = sorted(repo_details, key=lambda x: x['stargazers_count'], reverse=True)
    top_repos = sorted_repos[:top_n]
    return [repo['html_url'] for repo in top_repos]

Function for searching GitHub repositories

In [33]:
#7
def retrieve_files(repo_urls):
    file_names = ['README.md', 'install.md', 'package.json']
    files_content = {file_name: [] for file_name in file_names}

    github_api_key = os.getenv("GITHUB_API_KEY")  

    for url in repo_urls:
        if url.endswith('.git'):
            repo_name = url[:-4].split('github.com/')[-1]
        else:
            repo_name = url.split('github.com/')[-1]

        branches = ['main', 'master']
        for branch in branches:
            api_url = f"https://api.github.com/repos/{repo_name}/git/trees/{branch}?recursive=1"

            try:
                headers = {
                    'Authorization': f'Bearer {github_api_key}',
                    'Accept': 'application/vnd.github.v3+json'  
                }
                response = requests.get(api_url, headers=headers)
                
                if response.status_code == 200:
                    tree = response.json().get('tree', [])
                    for file_info in tree:
                        file_path = file_info['path']
                        file_name = file_path.split('/')[-1]
                        if file_name in file_names:
                            file_url = f"https://raw.githubusercontent.com/{repo_name}/{branch}/{file_path}"
                            file_response = requests.get(file_url)
                            if file_response.status_code == 200:
                                files_content[file_name].append(file_response.text)
                                print(f"Successfully retrieved {file_name} from {file_url}")
                            else:
                                print(f"Failed to retrieve {file_name} from {file_url}. Status code: {file_response.status_code}")
                    break
                elif response.status_code == 404:
                    print(f"Branch '{branch}' not found for repository {repo_name}.")
                else:
                    print(f"Failed to retrieve file list from {api_url}. Status code: {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error while trying to get file list from {api_url}: {e}")

    return files_content


Analyze and pre-process the data

In [8]:
#8
def parse_readme(content):
    sections = {}
    current_section = None
    lines = content.split('\n')

    for line in lines:
        header_match = re.match(r'^(#+)\s+(.*)', line)
        if header_match:
            section_name = header_match.group(2).strip()
            current_section = section_name
            sections[current_section] = []
            print(f"New section: {section_name}")
        elif current_section:
            sections[current_section].append(line)

    for section in sections:
        sections[section] = "\n".join(sections[section])

    print("Parsed sections:", sections)  
    return sections

In [9]:
#9
def parse_package_json(content):
    try:
        if not content.strip():
            print("Empty content for package.json.")
            return {}

        package_data = json.loads(content)
        return {
            "dependencies": package_data.get("dependencies", {}),
            "devDependencies": package_data.get("devDependencies", {}),
            "scripts": package_data.get("scripts", {})
        }
    except json.JSONDecodeError as e:
        print(f"Error parsing package.json: {e}")
        return {}

In [10]:
#10
def collect_context(files_content):
    context = {
        'README.md': [],
        'install.md': [],
        'package.json': []
    }

    for file_name, contents in files_content.items():
        if file_name == 'README.md':
            for content in contents:
                context[file_name].append(parse_readme(content))
        elif file_name == 'package.json':
            for content in contents:
                if content.strip():  
                    parsed_content = parse_package_json(content)
                    if parsed_content:  
                        context[file_name].append(parsed_content)
        else:
            context[file_name].extend(contents)  

    return context

In [11]:
#11
def extract_relevant_info(context):
    relevant_info = {
        "dependencies": [],
        "devDependencies": [],
        "scripts": [],
        "readme_sections": {}
    }

    for pkg in context.get('package.json', []):
        if "dependencies" in pkg:
            relevant_info["dependencies"].extend(pkg["dependencies"].keys())
        if "devDependencies" in pkg:
            relevant_info["devDependencies"].extend(pkg["devDependencies"].keys())

    for readme in context.get('README.md', []):
        for section, content in readme.items():
            if "setup" in section.lower() or "installation" in section.lower():
                relevant_info["readme_sections"][section] = content

    return relevant_info


In [12]:
#12
def format_input_for_model(relevant_info):
    input_data = {
        "prompt": "Based on the following information, generate a devcontainer.json file:\n\n"
    }

    input_data["prompt"] += "Dependencies:\n"
    input_data["prompt"] += ", ".join(relevant_info["dependencies"]) + "\n\n"

    input_data["prompt"] += "DevDependencies:\n"
    input_data["prompt"] += ", ".join(relevant_info["devDependencies"]) + "\n\n"

    input_data["prompt"] += "Scripts:\n"
    input_data["prompt"] += json.dumps(relevant_info["scripts"], indent=2) + "\n\n"

    input_data["prompt"] += "README Sections:\n"
    for section, content in relevant_info["readme_sections"].items():
        input_data["prompt"] += f"{section}:\n{content}\n\n"

    return input_data

Generate JSON file for Dev Container with OpenAI API

In [13]:
#13
def generate_dev_container_json(context, model="gpt-4o"):
    load_dotenv()

    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION")
    )

    system_prompt = "You are an AI assistant that creates JSON file for dev container.\n\n"
    user_prompt = f"Based on the following information, generate a devcontainer.json file:\n\n {context}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )


        if response.choices:
            for choice in response.choices:
                if choice.finish_reason == 'stop' and choice.message:
                    generated_json = choice.message.content.strip()
                    try:
            
                        start_index = generated_json.find('{')
                        end_index = generated_json.rfind('}') + 1
                        json_str = generated_json[start_index:end_index]
                        parsed_json = json.loads(json_str)

                        print("Generated Dev Container JSON Configuration:")
                        print(json.dumps(parsed_json, indent=4))
                        return parsed_json
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
                        return None
                else:
                    print(f"Invalid choice received: {choice}")
                    return None
        else:
            print("No valid choices found in API response.")
            return None

    except Exception as ex:
        print(f"Error generating devcontainer.json: {ex}")
        return None

Main execution block

In [38]:
#14
def main():
    prompt, flag = input_function()
    if flag == False:
        r = generate_github_repos(prompt)
        repos = get_top_repos(r)
        print(r)
        print(repos)
    else: 
        repos = prompt
    content = retrieve_files(repos)
    context = collect_context(content)
    relevant_content = extract_relevant_info(context)
    input = format_input_for_model(relevant_content)
    dev_container = generate_dev_container_json(input)
    return dev_container
output = main()

['https://github.com/angular/angular', 'https://github.com/angular/angular-cli', 'https://github.com/ngx-translate/core', 'https://github.com/ng-bootstrap/ng-bootstrap', 'https://github.com/angular/material', 'https://github.com/primefaces/primeng', 'https://github.com/ngrx/platform']
['https://github.com/angular/angular', 'https://github.com/angular/angular-cli', 'https://github.com/angular/material']
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.circleci/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.devcontainer/README.md
Successfully retrieved package.json from https://raw.githubusercontent.com/angular/angular/main/.github/actions/deploy-docs-site/package.json
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.vscode/README.md
Successfully retrieved README.md from https://raw.githubusercontent.com/angular/angular/main/.yarn/README.md
Su

Output

In [39]:
#15
print(output)

{'name': 'Angular Dev Container', 'image': 'mcr.microsoft.com/vscode/devcontainers/javascript-node:0-18', 'features': {}, 'customizations': {'vscode': {'extensions': ['angular.ng-template', 'dbaeumer.vscode-eslint', 'esbenp.prettier-vscode']}}, 'postCreateCommand': 'npm install', 'postStartCommand': 'npm run build', 'settings': {'terminal.integrated.shell.linux': '/bin/bash'}, 'mounts': ['source=${localWorkspaceFolder},target=/workspace,type=bind,consistency=cached'], 'forwardPorts': [4200], 'remoteUser': 'node', 'workspaceFolder': '/workspace', 'extensions': [{'name': 'node', 'version': 'latest'}], 'runArgs': ['--init']}
